In [4]:
# import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import unidecode
from itertools import chain, repeat



# load MLS roster data
mlsRosters = pd.read_csv('MLS Rosters 2017.csv')

# change name format in MLS roster file to [FIRST] [LAST], remove accents
for i in range(len(mlsRosters)):
    
    oldName = mlsRosters.at[i, 'Name']
    
    split = oldName.split(',')
    
    newName = str()
    
    if len(split) == 1: mlsRosters.at[i, 'Name'] = split[0]   
    else: newName = split[1].split(' ')[1] + ' ' + split[0]
        
    mlsRosters.at[i, 'Name'] = unidecode.unidecode(newName)

# split designations and create indicators
International = np.repeat(0, len(mlsRosters))
DP = np.repeat(0, len(mlsRosters))
youngDP = np.repeat(0, len(mlsRosters))
Homegrown = np.repeat(0, len(mlsRosters))
genAdidas = np.repeat(0, len(mlsRosters))

for i in range(len(mlsRosters)):
    
    if not isinstance(mlsRosters.at[i, 'Player Category'], float):
        
        designations = mlsRosters.at[i, 'Player Category'].split(', ')
        
        mlsRosters.at[i, 'Player Category'] = designations
        
        if 'INTL' in designations: International[i] = 1
        if 'DP' in designations: DP[i] = 1
        if 'Young DP' in designations: youngDP[i] = 1
        if 'HG' in designations: Homegrown[i] = 1
        if 'GA' in designations: genAdidas[i] = 1
            
mlsRosters['International'] = International
mlsRosters['DP'] = DP
mlsRosters['Young DP'] = youngDP
mlsRosters['Homegrown'] = Homegrown
mlsRosters['Gen. Adidas'] = genAdidas



# load FIFA data
fifa = pd.read_csv('FIFA 18 Ratings.csv',
                   low_memory = False)

# reduce dataset to MLS only
mlsTeams = ['LA Galaxy',
            'Toronto FC',
            'New York City Football Club',
            'Chicago Fire Soccer Club',
            'Portland Timbers',
            'Atlanta United FC',
            'Columbus Crew SC',
            'Seattle Sounders FC',
            'Orlando City Soccer Club',
            'San Jose Earthquakes',
            'New York Red Bulls',
            'Philadelphia Union',
            'Sporting Kansas City',
            'FC Dallas',
            'Minnesota United',
            'D.C. United',
            'New England Revolution',
            'Colorado Rapids',
            'Vancouver Whitecaps FC',
            'Houston Dynamo',
            'Real Salt Lake',
            'Montreal Impact']

fifa = fifa[fifa['Current Team'].isin(mlsTeams)]
fifa.index = range(len(fifa))

# remove accents
for i in range(len(fifa)):
    
    oldName = fifa.at[i, 'Name']
    
    fifa.at[i, 'Name'] = unidecode.unidecode(oldName)
    
    

# load MLS salary data
mlsSalaries = pd.read_csv('MLS Salaries 2017.csv')

# combine first and last name
name = []
for i in range(len(mlsSalaries)):
    
    if isinstance(mlsSalaries.at[i, 'First Name'], float): name.append(mlsSalaries.at[i, 'Last Name'])
    else: name.append(mlsSalaries.at[i, 'First Name'] + ' ' + mlsSalaries.at[i, 'Last Name'])
        
mlsSalaries['Name'] = name



# load clusters data
clusters = pd.read_csv('roles.csv')
clusters = clusters.rename(index = str, columns = {'Player' : 'Name', 'role' : 'Role'})

# assign names to clusters
clusters.loc[clusters['Role'] == 30, 'Role'] = 'CB'
clusters.loc[clusters['Role'] == 0, 'Role'] = 'FB'
clusters.loc[clusters['Role'] == 20, 'Role'] = 'DM'
clusters.loc[clusters['Role'] == 21, 'Role'] = 'CM'
clusters.loc[clusters['Role'] == 10, 'Role'] = 'AM'
clusters.loc[clusters['Role'] == 11, 'Role'] = 'CF'
clusters.loc[clusters['Role'] == 12, 'Role'] = 'FW'




# merge data frames
players = fifa[['Name',
                'Overall Rating',
                'Current Team',
                'Link']].merge(mlsRosters[['Name',
                                           'International',
                                           'DP',
                                           'Young DP',
                                           'Homegrown',
                                           'Gen. Adidas']],
                                       on = 'Name')

players = players.merge(mlsSalaries[['Name',
                                     'Base Salary',
                                     'Guaranteed Compensation']],
                        on = 'Name')

players = players.merge(clusters[['Name',
                                  'Role']],
                        on = 'Name')

In [5]:
lowMax = 65
highMin = 70

value = pd.DataFrame({'Role': list(set(players['Role']))})

lowInt = []
lowDom = []
lowValue = []

midInt = []
midDom = []
midValue = []

highInt = []
highDom = []
highValue = []

for i in range(len(value)):
    
    # all players at role
    posPlayers = players[players['Role'] == value.at[i, 'Role']]
    
    
    # low players at role
    lowPlayers = posPlayers[(posPlayers['Overall Rating'] < lowMax)]
    lowPlayers.index = range(len(lowPlayers))
    lowIntAll = lowPlayers.loc[lowPlayers['International'] == 1, 'Guaranteed Compensation']
    lowDomAll = lowPlayers.loc[lowPlayers['International'] == 0, 'Guaranteed Compensation']
    
    if len(lowIntAll) > 0: lowInt.append(min(lowIntAll))
    else: lowInt.append(np.nan)
        
    if len(lowDomAll) > 0: lowDom.append(min(lowDomAll))
    else: lowDom.append(np.nan)
        
    if lowInt[i] < lowDom[i]: lowValue.append(lowDom[i] - lowInt[i])
    else: lowValue.append(0)
        
    
    # mid players at role
    midPlayers = posPlayers[(posPlayers['Overall Rating'] > lowMax) & (posPlayers['Overall Rating'] < highMin)]
    midPlayers.index = range(len(midPlayers))
    midIntAll = midPlayers.loc[midPlayers['International'] == 1, 'Guaranteed Compensation']
    midDomAll = midPlayers.loc[midPlayers['International'] == 0, 'Guaranteed Compensation']
    
    if len(midIntAll) > 0: midInt.append(min(midIntAll))
    else: midInt.append(np.nan)
        
    if len(midDomAll) > 0: midDom.append(min(midDomAll))
    else: midDom.append(np.nan)
        
    if midInt[i] < midDom[i]: midValue.append(midDom[i] - midInt[i])
    else: midValue.append(0)
        
    
    # high players at role
    highPlayers = posPlayers[(posPlayers['Overall Rating'] > highMin)]
    highPlayers.index = range(len(highPlayers))
    highIntAll = highPlayers.loc[highPlayers['International'] == 1, 'Guaranteed Compensation']
    highDomAll = highPlayers.loc[highPlayers['International'] == 0, 'Guaranteed Compensation']
    
    if len(highIntAll) > 0: highInt.append(min(highIntAll))
    else: highInt.append(np.nan)
        
    if len(highDomAll) > 0: highDom.append(min(highDomAll))
    else: highDom.append(np.nan)
        
    if highInt[i] < highDom[i]: highValue.append(highDom[i] - highInt[i])
    else: highValue.append(0)
        

value['lowInt'] = lowInt
value['lowDom'] = lowDom
value['lowValue'] = lowValue

value['midInt'] = midInt
value['midDom'] = midDom
value['midValue'] = midValue

value['highInt'] = highInt
value['highDom'] = highDom
value['highValue'] = highValue

In [9]:
players.to_csv('playersFinal.csv', index = False)

In [6]:
value

,Role,lowInt,lowDom,lowValue,midInt,midDom,midValue,highInt,highDom,highValue
0,FB,53004.0,53000.00,0.00,52999.92,53004.00,4.08,205600.00,150000.00,0.00
1,CF,NaN,54075.00,0.00,65000.00,66150.00,1150.00,423000.00,654866.67,231866.67
2,AM,65004.0,54075.00,0.00,120000.00,105000.00,0.00,200000.04,180000.00,0.00
3,CM,53004.0,65000.04,11996.04,NaN,155000.00,0.00,5400000.00,137000.00,0.00
4,DM,NaN,65004.00,0.00,171250.00,65000.04,0.00,188337.33,350000.00,161662.67
5,FW,56379.0,53004.00,0.00,53000.00,65000.00,12000.00,212504.00,290000.00,77496.00
6,CB,NaN,54075.00,0.00,95000.04,75000.00,0.00,170000.04,65000.00,0.00
